In [12]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 
import talib as ta
# from numba import jit

from importlib import reload 
from qflib import basic,caldata
reload(basic)
reload(caldata)

global engine, conn
engine = basic.engine()
conn = basic.conn(engine)

df=pd.DataFrame()  # 初始化空值
vTest = 1000

data_dir = '../data/'
# sl_list = pd.read_csv( '../data/sl_hs_const.csv')
# sl_list = pd.read_csv( data_dir + 'sl_shengutong.csv')
# sl_list = pd.read_csv( '../data/sl_company.csv')
sl_list = pd.read_csv( data_dir + 'stocklist.csv')
# sl_list = pd.read_csv( data_dir+'data/stocklist.csv')
sl_list.head(3), sl_list.shape


''' 清空 stock_cal table '''
sql = "truncate table day_zhibiao"
basic.exec_sql(engine, sql)

In [13]:
def cal_pct_st():
    # 实体部分细化
    df['pct_upper'] = df.apply(caldata.cal_upper, axis=1)    # 上引线百分比
    df['pct_down']  = df.apply(caldata.cal_down,  axis=1)    # 下引线百分比
    df['pct_range'] = df.apply(caldata.cal_range, axis=1)    # 振幅百分比
    df['pct_st_range']  = df.apply(caldata.cal_st_range, axis=1)   # 实体部分百分比
    df['st_high']   = df.apply(caldata.cal_st_high,  axis=1)   # 实体高点价格
    df['st_low']    = df.apply(caldata.cal_st_low,   axis=1)   # 实体低点价格
    # df['pct_st']  = df.apply(caldata.cal_st, axis=1)   # 实体高度
    # 星线定义
    df['xing'] = df['pct_st_range']<=1
    return 'pct . '

def cal_macd():
    ''' way2 : ta-lib '''
    close = df.close.values  # 为 MACD 数据初始化close
    df['diff'], df['dea'], df['bar'] = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    return 'macd . '

def cal_ma():
    df.insert( df.shape[1], 'MA3', ta.MA(df['close'], 3) )
    df.insert( df.shape[1], 'MA5', ta.MA(df['close'], 5) )
    df.insert( df.shape[1], 'MA10', ta.MA(df['close'], 10) )
    df.insert( df.shape[1], 'MA20', ta.MA(df['close'], 20) )
    df.insert( df.shape[1], 'MA30', ta.MA(df['close'], 30) )
    df.insert( df.shape[1], 'MA60', ta.MA(df['close'], 60) )
    df.insert( df.shape[1], 'MA120', ta.MA(df['close'], 120) )
    df.insert( df.shape[1], 'MA250', ta.MA(df['close'], 250) )
    return 'ma . '

def cal_vol_ma():
    df['vol_ma3']=df['volume'].rolling(3).mean()
    df['vol_ma5']=df['volume'].rolling(5).mean()
    df['vol_ma10']=df['volume'].rolling(10).mean()
    df['vol_ma20']=df['volume'].rolling(20).mean()
    df['vol_ma60']=df['volume'].rolling(60).mean()
    return 'vol_ma . '
    
def cal_more():
    # # 股价变化均值
    # df.insert( df.shape[1], 'MA3_pct_change', ta.MA(df['pct_change'], 3) )
    # df.insert( df.shape[1], 'MA5_pct_change', ta.MA(df['pct_change'], 5) )
    # df.insert( df.shape[1], 'MA10_pct_change', ta.MA(df['pct_change'], 10) )
    # df.insert( df.shape[1], 'MA15_pct_change', ta.MA(df['pct_change'], 15) )
    # df.insert( df.shape[1], 'MA30_pct_change', ta.MA(df['pct_change'], 30) )
    # # N天之内最大股价
    # df['MAX3_pct_change']=df['pct_change'].rolling(3).max()
    # df['MAX5_pct_change']=df['pct_change'].rolling(5).max()
    # df['MAX10_pct_change']=df['pct_change'].rolling(10).max()
    # df['MAX15_pct_change']=df['pct_change'].rolling(15).max()
    # df['MAX30_pct_change']=df['pct_change'].rolling(30).max()
    # # n天之内最高收盘价
    # df['MAX90_high']=df['high'].rolling(90).max()
    # df['MAX30_high']=df['high'].rolling(30).max()
    # df['MAX3_high']=df['high'].rolling(3).max()
    
    ''' 计算BOLL '''
    # df['bb-u'], df['bb-m'], df['bb-l'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )
    # df['upper'], df['middle'], df['lower'] = ta.BBANDS( close,timeperiod=20,nbdevup=2,nbdevdn=2,matype=0 )
    


# import
# df['MA15_pct_change'] = df['pct_change'].rolling(15).mean() 


In [14]:
%%time

print('计算各类指标 : ')
for i in range( len(sl_list)  ):
    
    # begin with ts_code
    print(sl_list.index[i],'-', sl_list['ts_code'][i], sl_list['name'][i], end=" ")
    
    # 不计算8字头，北京股票
    if sl_list['ts_code'][i].startswith('8'):
        print( 'ignore 8xxx', end='\n' )
        continue
    # 不计算688字头，科创板
    if sl_list['ts_code'][i].startswith('688'):
        print( 'ignore 688x', end='\n' )
        continue
    
    # 本地MySQL读取
    sql = "select * from tran_daily where ts_code='"+sl_list['ts_code'][i]+"'"
    df = basic.read_data(conn, sql)
    df.set_index('trade_date', drop=True, append=False, inplace=True)
    df = df.sort_index(ascending=True)
    # 如果没有交易记录，则跳过loop
    if len(df) == 0:
        print( 'no data', end='' )
        continue  
    
    # 计算 percentage 和 实体部分
    print( cal_pct_st(), end='' )
           
    # 计算macd
    print( cal_macd(), end='' )
    
    # 计算ma
    print( cal_ma(), end='' )

    # 计算vol_ma
    print( cal_vol_ma(), end='' )

    # 空值填充为0
    df = df.fillna(value=0)

    """ 输出 to csv """
    # df.to_csv( data_dir+'cal/' + sl_list['ts_code'][i] +'.cal.csv', index=True )    
    ''' 输出到 mysql '''
    sql = "delete from day_zhibiao where ts_code='"+sl_list['ts_code'][i] +"'"
    basic.exec_sql(conn, sql)
    # df.to_sql('day_zhibiao', conn, index=True, if_exists='append', chunksize=2000)
    write_no = df.to_sql('day_zhibiao', conn, index=True, if_exists='append', chunksize=2000) # 20000个，每天成交几千只股票，足够了
    # index=True, 需要输出日期字段    
    # 首次运行，建立stock_cal表, 可以从df中建立表，表之前不存在，然后再改字段和index

    print( end='\n')

print('\n\n指标运算结束')



计算各类指标 : 
0 - 601318.SH 中国平安 pct . macd . ma . vol_ma . 
1 - 300474.SZ 景嘉微 pct . macd . ma . vol_ma . 


指标运算结束
CPU times: total: 31.2 ms
Wall time: 407 ms


In [15]:
# basic.write_data(engine, df, 'stock_basic', False, 'append')
# basic.write_data(engine, df, 'stock_cal', True, 'replace')

In [16]:

# 插入和删除行，很有用
# df.insert( df.shape[1], 'MA250', ta.MA(df['close'], 250) )
# df.drop( labels=['MA125'], axis=1, inplace=True)